In [1]:
# Imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# to enable local imports
module_path = os.path.abspath('../../code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from machine_learning_load_data import loadOnlineEEGdata

Using TensorFlow backend.


D:\Masterthesis\thesis_eeg\code


In [2]:
tf.__version__

'2.1.0'

In [3]:
keras.__version__

'2.2.4-tf'

In [2]:
# Load some online EEG Data
eegData, freqData, entropyData = loadOnlineEEGdata(dirPath='../../../EEG_Data/eeg_data_online', splitData=True)

X_train_eeg, y_train_eeg, X_test_eeg, y_test_eeg = eegData
X_train_freq, y_train_freq, X_test_freq, y_test_freq = freqData
X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy = entropyData

# reshape
#X_train_freq = X_train_freq.reshape(X_train_freq.shape[0], X_train_freq.shape[2])
#X_test_freq = X_test_freq.reshape(X_test_freq.shape[0], X_test_freq.shape[2])

#X_train_entropy = X_train_entropy.reshape(X_train_entropy.shape[0], X_train_entropy.shape[2])
#X_test_entropy = X_test_entropy.reshape(X_test_entropy.shape[0], X_test_entropy.shape[2])


Loading Online EEG Data from ../../../EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)
Entropy Data Shape:
(5024, 1, 200) (5024,) (2154, 1, 200) (2154,)


In [15]:
def TrainEvaluateNet(x_train, y_train, x_test, y_test):

    #Convert data set to numpy array of type float32
    #x_train = np.asarray(x_train, np.float32)
    #y_train = np.asarray(y_train, np.float32)
    #x_test = np.asarray(x_test, np.float32)
    #y_test = np.asarray(y_test, np.float32)

    #Number of neurons
    neurons_num = int((2/3) * x_train.shape[2])

    #Initialize neural net
    #output of layer = keras.layers.Dense(number of neurons, activation function, name)(input of layer)
    inputs = keras.Input(shape=(x_train.shape[1], x_train.shape[2]), name='input_points')
    x = keras.layers.Dense(neurons_num, activation='relu', name='L1')(inputs)
    x = keras.layers.Dense(neurons_num, activation='relu', name='L2')(x)
    outputs = keras.layers.Dense(2, activation='softmax', name='output_point')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='fatigue_classifier')

    #Set optimizer, loss function and metrics to track
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #binary_crossentropy

    #Summary of the neural net
    #model.summary()
    
    #Train neural net
    model.fit(x_train, y_train, epochs=3)

    #Evaluate neural net
    model.evaluate(x_test, y_test)

    #Predict best route
    return model

In [76]:
def convertDataSet(dataset, channels):
    #converts the frequency dataset from (1008,1,1200) (samples, placeholder, channels + values) in (1008,40,30,1) (samples, channels, values, placeholder)
    newdataset = np.zeros((int(dataset.shape[0]), int((dataset.shape[2]/channels)), int(channels), 1))
    for sample in range(dataset.shape[0]):
        for channel in range(0, dataset.shape[2], channels):
            for values in range(channels):
                newdataset[sample, int(channel/30), values, 0] = dataset[sample, 0, channel + values]
    
    newdataset = np.asarray(newdataset, np.float32)
    return newdataset


def TrainEvaluateNet(x_train, y_train, x_test, y_test):

    #converting training dataset
    x_train = convertDataSet(x_train, 30)
    x_test = convertDataSet(x_test, 30)
    
    #Settings
    batch_size = 64
    num_classes = 2
    epochs = 3
    neurons_num = int((2/3) * x_train.shape[2]) #Number of neurons

    #Initialize neural net
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(neurons_num*500, activation='relu'))
    model.add(Dense(neurons_num*100, activation='relu'))
    model.add(Dense(neurons_num*10, activation='relu'))
    model.add(Dense(neurons_num, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes-1, activation='sigmoid', name='last_dense')) # 'softmax'

    #model.summary()
    
    #Set optimizer, loss function and metrics to track
    #model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    model.compile(loss=keras.losses.squared_hinge, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    #Summary of the neural net
    #model.summary()
    
    #Train neural net
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
#          validation_split=0.2)

    #Evaluate neural net
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    #Predict best route
    return model

In [16]:
y_train_freq.shape

(1008,)

In [56]:
X_train_freq.shape

(1008, 1, 1200)

In [77]:
def ApplyNet(input_data, model):
    prediction = model.predict([input_data]) 
    return prediction

model = TrainEvaluateNet(X_train_freq, y_train_freq, X_test_freq, y_test_freq)
#ApplyNet(model)


Train on 1008 samples, validate on 432 samples
Epoch 1/3
1008/1008 [==============================] - 40s 40ms/step - loss: 1.2952 - accuracy: 0.4970 - val_loss: 1.1351 - val_accuracy: 0.4444
Epoch 2/3
1008/1008 [==============================] - 41s 40ms/step - loss: 1.2911 - accuracy: 0.5367 - val_loss: 1.1324 - val_accuracy: 0.4444
Epoch 3/3
1008/1008 [==============================] - 44s 44ms/step - loss: 1.2883 - accuracy: 0.5238 - val_loss: 1.1297 - val_accuracy: 0.4444
Test loss: 1.129665274310995
Test accuracy: 0.4444444477558136


In [74]:
#model = TrainEvaluateNet(X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy)
#ApplyNet(model)
